# Draft:  Binary fitting using AMICAL

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data
- *import amical* University of Sydney interferometric package


Clone AMICAL repository on your computer:
git clone https://github.com/SydneyAstrophotonicInstrumentationLab/AMICAL.git  

Then, in the astropy conda environment, install AMICAL:
cd AMICAL/
pip install .


In [ ]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import amical

## Introduction
This notebook takes a calibrated OIFITS file and estimates the parameters of a best-fit binary model  It is closely based on Anthony Soulain's example script analyzing NIRISS AMI commissioning on the know binary AB Dor AC. 

### Defining terms
[OIFITS](https://doi.org/10.1051/0004-6361/201526405) files are the Optical Interferometry standard FITS files used by the community.  AMICAL is **A**perture **M**asking **I**nterferometry **C**alibration and **A**nalysis **L**ibrary.  It is developed by the Sydney Astrophotonic Instrumentation Lab [![version](http://img.shields.io/badge/AMICAL-v0.5dev-orange.svg?style=flat)](https://github.com/SydneyAstrophotonicInstrumentationLab/AMICAL.git)


A science target's observables are **calibrated** by the observables of a PSF calibrator star's image, to remove telescope/instrument effects (as far as possible).


***

## Loading data
Load the input, a science target's calibrated oifits file, from **longURL**.  This file needs to have interferometric observables corresponding to all possible baselines in the non-redundant mask, and their "straight" standard deviations (without consideration for linear dependence within a set of variables).

For example, if we wanted to query for data from MAST for Kepler we might do something like:

**Developer note** Use our tools to look at some of the fits tables: closure phase and fringe amplitude in particlar.  More visuals during the reduction later on here...

The example dataset comes with a NIRISS simulated dataset representing a faint binary star 
- Magnitude difference: 6 mag at 4.3 μm
- Sub-λ/D separation 147.7 mas
- Position angle theta 46.6 degrees 
- Filter F430M


In [ ]:
sep = 147.7  # binary separation [mas]
theta = 46.6  # position angle (pa) [deg]
dm = 6.0  # contrast ratio [mag

In [ ]:
# Download the products
Observations.download_products(yourProd, mrp_only=False, cache=False)

## File information

Explain pertinent details about the file you've just downloaded. For example, if working with Kepler lightcurves, explain what's in the different file extensions:

```
- No. 0 (Primary): This HDU contains meta-data related to the entire file.
- No. 1 (Light curve): This HDU contains a binary table that holds data like flux measurements and times. We will extract information from here when we define the parameters for the light curve plot.
- No. 2 (Aperture): This HDU contains the image extension with data collected from the aperture. We will also use this to display a bitmask plot that visually represents the optimal aperture used to create the SAP_FLUX column in HDU1.
```

## Sections (xN)

The meat of your notebook should be sub-divided into sections with useful names to whatever extent makes sense base on the content. Break sections up with standard Markdown syntax headings:

```
# Section 1

Intro to section 1

### Sub-section 1a

More detailed info about Section 1

## Section 2

A complete thought that's as important as section 1 but doesn't need sub-sections.
```

## Exercises
Most notebooks are trying to convey _some_ information to their reader. It's often a good idea, if possible, to include in the notebook something where there is not an included answer so that the reader can cement in their mind how whatever it is they were supposed to learn from that notebook. If you do have exercise(s), be sure to leave a blank code cell underneath to show the user that it's meant for them to try it out right there. You may also want to include a "solutions" notebook next to your main notebook for the user to look at after they have finished their attempt.

## Aditional Resources
While this isn't always necessary, sometimes you want to provide some more resources for the reader who wants to learn something beyond what's in the notebook. Sometimes these don't exist, but if they do, it's good to put them at the end to give the reader somewhere else to go. Usually a list of links using markdown bullet-plus-link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)

## About this notebook
Let the world know who the author of this great notebook is! If possible/appropriate, include a contact email address for users who might need support (e.g. archive@stsci.edu)

**Author:** Anand Sivaramakrishnan's transcription of Anthony Soulain's analysis script.  
**Updated On:** YYYY-MM-DD

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 